# <span style='color:#ff5f27'> Initialization </span>

### Hopsworks Settings

In [ ]:
import sys
from pathlib import Path
import os

def is_google_colab() -> bool:
    if "google.colab" in str(get_ipython()):
        return True
    return False

def clone_repository() -> None:
    !git clone https://github.com/featurestorebook/mlfs-book.git
    %cd mlfs-book

def install_dependencies() -> None:
    !pip install --upgrade uv
    !uv pip install --all-extras --system --requirement pyproject.toml


if is_google_colab():
    clone_repository()
    install_dependencies()
    root_dir = str(Path().absolute())
    print("Google Colab environment")
else:
    root_dir = Path().absolute()
    # Strip ~/notebooks/ccfraud from PYTHON_PATH if notebook started in one of these subdirectories
    if root_dir.parts[-1:] == ('airquality',):
        root_dir = Path(*root_dir.parts[:-1])
    if root_dir.parts[-1:] == ('notebooks',):
        root_dir = Path(*root_dir.parts[:-1])
    root_dir = str(root_dir) 
    print("Local environment")

# Add the root directory to the `PYTHONPATH` to use the `recsys` Python module from the notebook.
if root_dir not in sys.path:
    sys.path.append(root_dir)
print(f"Added the following directory to the PYTHONPATH: {root_dir}")
    
# Read the API keys and configuration variables from the file <root_dir>/.env
from mlfs import config
if os.path.exists(f"{root_dir}/.env"):
    settings = config.HopsworksSettings(_env_file=f"{root_dir}/.env")

### Imports

In [ ]:
import datetime
import pandas as pd
from xgboost import XGBRegressor
import hopsworks
import json
from mlfs import util

# <span style='color:#ff5f27'> Retrieve Data </span>

### Retrieve Metadata

In [ ]:
today = datetime.datetime.now() - datetime.timedelta(0)
tomorrow = today + datetime.timedelta(days = 1)
today

In [ ]:
# Check if HOPSWORKS_API_KEY env variable is set or if it is set in ~/.env
if settings.HOPSWORKS_API_KEY is not None:
    api_key = settings.HOPSWORKS_API_KEY.get_secret_value()
    os.environ['HOPSWORKS_API_KEY'] = api_key
project = hopsworks.login()
fs = project.get_feature_store() 

secrets = hopsworks.get_secrets_api()
location_str = secrets.get_secret("BIKES_LOCATION_JSON").value
location = json.loads(location_str)

country=location['country']
city=location['city']
latitude=location['latitude']
longitude=location['longitude']
today = datetime.date.today()
yesterday = today - datetime.timedelta(days=1)

print(f"City: {city}")
print(f"Contry: {country}")
print(f"Latitude: {latitude}")
print(f"Longitude: {longitude}")
print(f"Day: {today}")

# <span style='color:#ff5f27'> Retrieve the Model </span>

### Download the Model From Model Registry

In [ ]:
mr = project.get_model_registry()

retrieved_model = mr.get_model(
    name="trento_bikes_xgboost_model",
    version=1,
)

fv = retrieved_model.get_feature_view()

# Download the saved model artifacts to a local directory
saved_model_dir = retrieved_model.download()

### Loading the XGBoost Regressor Model

In [ ]:
retrieved_xgboost_model = XGBRegressor(enable_categorical=True)
retrieved_xgboost_model.load_model(saved_model_dir + "/trento_bikes_xgboost_model.json")

# Displaying the retrieved XGBoost regressor model
retrieved_xgboost_model

# <span style='color:#ff5f27'> Retrieve Data </span>

### Get Weather Forecast

In [ ]:
weather_fg = fs.get_feature_group(
    name='weather_trento',
    version=1,
)

# To avoid hourly discrepancies, we filter dates > than yesterday (rather than >= today)
weather_data = weather_fg.filter(weather_fg.date > today-datetime.timedelta(days=1)).read()

weather_data = weather_data.sort_values(by=['date'], ignore_index=True)
weather_data['days_before_forecast'] = range(1, len(weather_data)+1)
weather_data.head(10)

### Get Today's Bikes

In [ ]:
bikes_fg = fs.get_feature_group(
    name='bikes_trento',
    version=1,
)

bikes_today = bikes_fg.filter(bikes_fg.date > (today-datetime.timedelta(days=1))).read()
bikes_today.head()

In [ ]:
bikes_today.info()

# <span style='color:#ff5f27'> Making the Predictions </span>

### Prepare the Data to Fed to the Model

In [ ]:
# Just in case there are more rows retrieved
bikes_today = bikes_today.sort_values(by=['date'], ascending=False)

joint_df = pd.merge(
    bikes_today[['id', 'city']],
    weather_data[['date', 'weather_code', 'apparent_temperature_mean', 'daylight_duration', 'precipitation_sum', 'wind_speed_10m_max', 'city', 'days_before_forecast']],
    on='city', how='left'
)
joint_df = joint_df.drop(columns=['city'])
joint_df['id'] = joint_df['id'].astype("category")
joint_df['weather_code'] = joint_df['weather_code'].astype("category")

joint_df.head(10)


In [ ]:
joint_df.info()

### Make the Predictions

In [ ]:
joint_df['predicted_bikes'] = retrieved_xgboost_model.predict(
    joint_df[['id', 'weather_code', 'apparent_temperature_mean', 'daylight_duration', 'precipitation_sum', 'wind_speed_10m_max']]
)

joint_df['city'] = city
joint_df['country'] = country
joint_df = joint_df.sort_values(by=['date'])

joint_df.head()

In [ ]:
joint_df.info()

# <span style='color:#ff5f27'> Create Forecast Graph </span>

### Obtain Metadata 

In [ ]:
metadata = pd.DataFrame()
metadata['id'] = joint_df['id'].drop_duplicates()  
metadata.head()

### Plot the Forecasts

In [ ]:
for i in range(2):#range(metadata.shape[0]):
    name = metadata.iloc[i]['id']
    pred_file_path = f"{root_dir}/dashboard/img/bikes_forecast_"+name+".png"
    joint_df_i = joint_df[joint_df['id'] == name]
    plt = util.plot_bikes_forecast(city, name, joint_df_i, pred_file_path)
    plt.close()

# <span style='color:#ff5f27'> Upload Monitor Feature Group </span>

In [ ]:
monitor_fg = fs.get_or_create_feature_group(
    name='bikes_predictions_trento',
    description='Bike prediction monitoring for Trento',
    version=1,
    primary_key=['city','id','date','days_before_forecast'],
    event_time="date"
)

Convert `weather_code` to integer as Hopsworks doesn't support categories

In [ ]:
joint_df['weather_code'] = joint_df['weather_code'].astype('int32')

In [ ]:
monitor_fg.insert(joint_df, wait=True)

In [ ]:
monitor_fg.update_feature_description("id", "ID of the station")
monitor_fg.update_feature_description("date", "Measurement Day")
monitor_fg.update_feature_description("weather_code", "Weather Condition Numeric Code (WMO)")
monitor_fg.update_feature_description("apparent_temperature_mean", "Apparent Temperature, Mean of the Day")
monitor_fg.update_feature_description("daylight_duration", "Seconds of Sun for the Day")
monitor_fg.update_feature_description("precipitation_sum", "Total Precipitation for the Day")
monitor_fg.update_feature_description("wind_speed_10m_max", "Wind Speed")
monitor_fg.update_feature_description("days_before_forecast", "Days before the forecast")
monitor_fg.update_feature_description("predicted_bikes", "Number of available bikes by the model")
monitor_fg.update_feature_description("city", "City of Measurement")
monitor_fg.update_feature_description("country", "Country of Measurement")

# <span style='color:#ff5f27'> Create Hindcast </span>

### Retrieve Monitor Data

In [ ]:
# We will create a hindcast chart for only the forecasts made 1 day beforehand
monitoring_df = monitor_fg.filter(monitor_fg.days_before_forecast == 1).read()
monitoring_df.head()

### Retrieve Bikes Data

In [ ]:
bikes_fg = fs.get_feature_group(name='bikes_trento', version=1)
bikes_df = bikes_fg.read()
bikes_df.head()

### Merge the Two

In [ ]:
outcome_df = bikes_df[['date', 'bikes', 'id']]
outcome_df['date'] = outcome_df['date'].apply(lambda x: pd.Timestamp(x).replace(hour=0, minute=0, second=0, microsecond=0))
preds_df =  monitoring_df[['date', 'predicted_bikes', 'id']]

hindcast_df = pd.merge(preds_df, outcome_df, on=['date','id'])
hindcast_df = hindcast_df.sort_values(by=['date'])
hindcast_df

# If there are no outcomes for predictions yet, generate some predictions/outcomes from existing data
if len(hindcast_df) == 0:
    print("Using Prediction Backfill")
    hindcast_df = util.backfill_predictions_for_monitoring_bikes(weather_fg, bikes_df, monitor_fg, retrieved_xgboost_model)
    hindcast_df = hindcast_df[['date', 'predicted_bikes', 'id', 'bikes']]

hindcast_df.head(10)

In [ ]:
hindcast_df.info()

### Plot Hindcast

In [ ]:
for i in range(metadata.shape[0]):
    name = metadata.iloc[i]['id']
    hindcast_file_path = f"{root_dir}/dashboard/img/bikes_hindcast_1day_"+name+".png"
    hindcast_df_i = hindcast_df[hindcast_df['id'] == name]
    plt = util.plot_bikes_forecast(city, name, hindcast_df_i, hindcast_file_path, hindcast=True)
    plt.close()

# <span style='color:#ff5f27'> Upload Pictures </span>

In [ ]:
dataset_api = project.get_dataset_api()
str_today = today.strftime("%Y-%m-%d")
folder_name = "bikes_v1"

if dataset_api.exists(f"Resources/{folder_name}") == False:
    dataset_api.mkdir(f"Resources/{folder_name}")

for i in range(metadata.shape[0]):
    name = metadata.iloc[i]['id']

    pred_file_path = f"{root_dir}/dashboard/img/bikes_forecast_"+name+".png"
    hindcast_file_path = f"{root_dir}/dashboard/img/bikes_hindcast_1day_"+name+".png"

    dataset_api.upload(pred_file_path, f"Resources/{folder_name}/{city}_{name}_{str_today}", overwrite=True)
    dataset_api.upload(hindcast_file_path, f"Resources/{folder_name}/{city}_{name}_{str_today}", overwrite=True)

proj_url = project.get_url()
print(f"See images in Hopsworks here: {proj_url}/settings/fb/path/Resources/{folder_name}")